# Проект для «Викишоп»
Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.


**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [2]:
import pandas as pd
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

/tmp/ipykernel_130/446590151.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Знакомство с данными

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
display(data.head())
print(len(data))

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


159571


In [21]:
print(data['toxic'].describe())

count    159571.000000
mean          0.101679
std           0.302226
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: toxic, dtype: float64


#### Комментарий
Классы представлены в дисбалансе, токсичных комментариев ~ 10% от общего числа.

### Лемматизация и удаление лишних символов

In [4]:
def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(text)
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word_list])
    return lemmatized_output

In [5]:
def clear_text(text):
    return ' '.join(re.sub(r'[^a-zA-Z ]', ' ', text).split())

In [6]:
def lemmatize_and_clear(text):
    lemm_text = lemmatize(clear_text(text))
    return lemm_text

In [7]:
%%time
data['lemm_text'] = data['text'].progress_apply(lambda x: lemmatize_and_clear(x))


CPU times: user 2min 49s, sys: 2.36 s, total: 2min 51s
Wall time: 3min


In [8]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


#### Комментарий
Текст был пролемматизирован и очищен от лишних символов.

#### Сохранение лемматизированного текста в csv файл 

In [9]:
data.to_csv ('data_lemms.csv', index = False, header = True)

In [22]:
del data 

In [5]:
data_lemms = pd.read_csv('data_lemms.csv')

In [6]:
data_lemms.dropna(inplace = True)
data_lemms.head(5)

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He match this background colour I m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


### Подготовка признаков и разбиение на выборки

In [7]:
corpus = data_lemms['lemm_text'].values
target = data_lemms['toxic']

In [9]:
del data_lemms

In [10]:
corpus_train, corpus_test, target_train, target_test = train_test_split(corpus,target, test_size = 0.2, random_state = 12345)

Произведем векторизацию текстов обучающей выборки.

In [11]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords, min_df = 0.0001)
count_tf_idf.fit(corpus_train)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=0.0001, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary=None)

In [12]:
features_train = count_tf_idf.transform(corpus_train)
features_test = count_tf_idf.transform(corpus_test)

In [13]:
print("Размер матрицы:", features_train.shape)

Размер матрицы: (127651, 15097)


#### Комментарий
При составлении  матрицы состоящей из TF-IDF, я старался избавиться от большого числа  редко встречаемых слов. При помощи параметра min_df. Выбранное значение позволило уменьшить кол-во признаков, которые используются для обучения моделей. Тестовая выборка составила 0.2 всех значений.

## Обучение

Для выполнения проекта использовались линейные модели, т.к. они обладают высокой скоростью обучения.

### LogisticRegression

In [108]:
params = {
    'C':[0.1,1,10,100]
}

In [109]:
%%time
log_reg_model = GridSearchCV(LogisticRegression(random_state = 12345), params, cv = None)
log_reg_model.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


CPU times: user 1min 28s, sys: 1min 26s, total: 2min 55s
Wall time: 2min 55s


GridSearchCV(cv=None, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=12345, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None, param_grid={'C': [0.1, 1, 10, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [110]:
log_reg_model.best_estimator_

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=12345, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [111]:
predictions = log_reg_model.predict(features_test)

In [112]:
print('F1-мера на обучающей выборке: {:.3f}'.format(f1_score(target_train, log_reg_model.predict(features_train))))
print('F1-мера на тестовой выборке: {:.3f}'.format(f1_score(target_test, predictions)))

F1-мера на обучающей выборке: 0.859
F1-мера на тестовой выборке: 0.770


#### Комментарий
Полученная F1-мера на тестовой выборке удоволятворяет условиям задачи. Но по скорости обучения уступает двум следующим моделям.

###  RidgeClassifier

In [90]:
params = {
    'alpha':[0.01,0.1,1,10,100]
}

In [91]:
%%time
ridge_model = GridSearchCV(RidgeClassifier(random_state = 12345), params, cv = None)
ridge_model.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 48.6 s, sys: 0 ns, total: 48.6 s
Wall time: 48.8 s


GridSearchCV(cv=None, error_score='raise-deprecating',
             estimator=RidgeClassifier(alpha=1.0, class_weight=None,
                                       copy_X=True, fit_intercept=True,
                                       max_iter=None, normalize=False,
                                       random_state=12345, solver='auto',
                                       tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [0.01, 0.1, 1, 10, 100]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [92]:
ridge_model.best_estimator_

RidgeClassifier(alpha=1, class_weight=None, copy_X=True, fit_intercept=True,
                max_iter=None, normalize=False, random_state=12345,
                solver='auto', tol=0.001)

In [93]:
predictions_ridge = ridge_model.predict(features_test)

In [94]:
print('F1-мера на обучающей выборке: {:.3f}'.format(f1_score(target_train, ridge_model.predict(features_train))))
print('F1-мера на тестовой выборке: {:.3f}'.format(f1_score(target_test, predictions_ridge)))

F1-мера на обучающей выборке: 0.757
F1-мера на тестовой выборке: 0.703


#### Комментарий
F1-мера не удоволятворяет условиям задачи, но скорость обучения выше, чем у логистической регрессии.

### PassiveAggressiveClassifier

In [95]:
params = {
    'C':[0.01,0.05,0.1,1,10]
}

In [96]:
%%time
pas_agr_model = GridSearchCV(PassiveAggressiveClassifier(random_state = 12345), params, cv = None)
pas_agr_model.fit(features_train, target_train)

/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


CPU times: user 12.5 s, sys: 0 ns, total: 12.5 s
Wall time: 12.6 s


GridSearchCV(cv=None, error_score='raise-deprecating',
             estimator=PassiveAggressiveClassifier(C=1.0, average=False,
                                                   class_weight=None,
                                                   early_stopping=False,
                                                   fit_intercept=True,
                                                   loss='hinge', max_iter=1000,
                                                   n_iter_no_change=5,
                                                   n_jobs=None,
                                                   random_state=12345,
                                                   shuffle=True, tol=0.001,
                                                   validation_fraction=0.1,
                                                   verbose=0,
                                                   warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.05, 0.1, 1,

In [97]:
pas_agr_model.best_estimator_

PassiveAggressiveClassifier(C=0.05, average=False, class_weight=None,
                            early_stopping=False, fit_intercept=True,
                            loss='hinge', max_iter=1000, n_iter_no_change=5,
                            n_jobs=None, random_state=12345, shuffle=True,
                            tol=0.001, validation_fraction=0.1, verbose=0,
                            warm_start=False)

In [98]:
predictions_pacl = pas_agr_model.predict(features_test)

In [99]:
print('F1-мера на обучающей выборке: {:.3f}'.format(f1_score(target_train, pas_agr_model.predict(features_train))))
print('F1-мера на тестовой выборке: {:.3f}'.format(f1_score(target_test, predictions_pacl)))

F1-мера на обучающей выборке: 0.830
F1-мера на тестовой выборке: 0.781


#### Комментарий
Данная модель показала наивысшую F1-меру и скорость обучения.

## Выводы

В ходе выполнения проекта текст комментариев был лемматизирован и очищен от лишних слов, затем была получена матрица с TF-IDF слов. На основании этой матрицы были обучены модели. Наилучший показатель по качеству прогноза и скорости обучения показала модель PassiveAggressiveClassifier. В проекте не использовались другие модели(случ. леса или град.бустинга деревьев), из-за долгого времени их обучения, возможно их время обучения можно было сократить, при уменьшении матрицы с TF-IDF, но я это пока не проверял). 